Entrenando un modelo con PCA  

Considerando xtrain, ytrain definido en las céldas de arriba, crea dos modelos:  

Un pipeline considerando PCA con n_components=0.7 seguido de un modelo KNeighborsClassifier con n_neighbors=5  
KNeighborsClassifier con n_neighbors=5  
Realiza un cross validation con n_components=3.  

¿Qué resultados arroja cada modelo?
¿que ventajas tiene cada modelo?
¿Cuánto tarda en correr cada cross-validation?

In [2]:
#pip install --upgrade pip
#pip install tensorflow

from numpy.linalg import svd
# conda install tensorflow
from tensorflow.keras import datasets
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, cross_validate

In [3]:
train, test = datasets.mnist.load_data()
xtrain, ytrain = train
xtest, ytest = test

In [4]:
Ntr, M1tr, M2tr = xtrain.shape
Nte, M1te, M2te = xtest.shape

PRIMERO EXPLOREMOS CON EL PIPELINE HACIENDO REDUCCIÓN DE DIMENSIÓN Y KNC

In [5]:
model = Pipeline((
 ("PCA", PCA(n_components=0.7)),
 ("KNC", KNeighborsClassifier(n_neighbors=5))   
 ))

In [8]:
%%time
fit=model.fit(xtrain.reshape(Ntr, -1), ytrain.reshape(Ntr, -1))
print(f"accuracy: {accuracy_score(ytest.reshape(Nte, -1), fit.predict(xtest.reshape(Nte, -1)))}")

C:\Users\migro\Anaconda3\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


accuracy: 0.9738
Wall time: 22.7 s


EXPLORAMOS SIN REDUCCIÓN DE DIMENSIÓN

In [52]:
%%time
KNC=KNeighborsClassifier(n_neighbors=5)
fit2=KNC.fit(xtrain.reshape(Ntr, -1), ytrain.reshape(Ntr, -1))
print(f"accuracy: {accuracy_score(ytest.reshape(Nte, -1), fit2.predict(xtest.reshape(Nte, -1)))}")

C:\Users\migro\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


accuracy: 0.9688
Wall time: 29min 16s


REALIZAMOS CROSSVALIDATION CON CV=3 PARA EL PIPE

In [54]:
%%time
cv_pipe= cross_validate(model, xtrain.reshape(Ntr, -1), ytrain.reshape(Ntr, -1), cv=3, scoring=["r2", "neg_mean_absolute_error"], n_jobs=-1)
r2_pipe = cv_pipe["test_r2"]
mae_pipe = -cv_pipe["test_neg_mean_absolute_error"]
print(f"mean r2: {r2_pipe.mean():0.2f}")
print(f"mean mae: {mae_pipe.mean():,.2f}")

mean r2: 0.94
mean mae: 3.78
Wall time: 1min 16s


REALIZAMOS CROSSVALIDATION CON CV=3 SIN REDUCCIÓN DE DIMENSIÓN

In [10]:
%%time
cv= cross_validate(KNC, xtrain.reshape(Ntr, -1), ytrain.reshape(Ntr, -1), cv=3, scoring=["r2", "neg_mean_absolute_error"], n_jobs=-1)
r2 = cv["test_r2"]
mae = -cv["test_neg_mean_absolute_error"]
print(f"mean r2: {r2.mean():0.2f}")
print(f"mean mae: {mae.mean():,.2f}")

mean r2: 0.93
mean mae: 4.94
Wall time: 18min 59s


¿Qué resultados arroja cada modelo?  

KNeighbors con reducción de dimensión:  
accuracy: 0.9738  
Wall time: 22.7 s  

KNeighbors sin reducción de dimensión:  
accuracy: 0.9688  
Wall time: 29min 16s  
    
CV: KNeighbors con reducción de dimensión:  
mean r2: 0.94  
mean mae: 3.78  
Wall time: 1min 16s    

CV: KNeighbors sin reducción de dimensión:  
mean r2: 0.93  
mean mae: 4.94  
Wall time: 18min 59s  

¿Qué ventajas tiene cada modelo?

En ambos casos tanto al hacer CV como no, la reducción de dimensiones nos ayuda en tiempo computacional de forma abismal.
Además la precisión es muy parecida en ambos experimentos

¿Cuánto tarda en correr cada cross-validation?  

CV: KNeighbors con reducción de dimensión:  
mean r2: 0.94  
mean mae: 3.78  
Wall time: 1min 16s    

CV: KNeighbors sin reducción de dimensión:  
mean r2: 0.93  
mean mae: 4.94  
Wall time: 18min 59s  
